In [ ]:
import pickle
import sys
sys.path.append("../")
from Join_scheme.data_prepare import process_stats_data
from BayesCard.Models.Bayescard_BN import Bayescard_BN
import time
import pandas as pd
import numpy as np
from BayesCard.Evaluation.cardinality_estimation import parse_query_single_table

In [ ]:
import pickle
import time
import numpy as np
from Join_scheme.data_prepare import process_stats_data
from Join_scheme.bound import Bound_ensemble
from BayesCard.Models.Bayescard_BN import Bayescard_BN
from BayesCard.Evaluation.cardinality_estimation import parse_query_single_table


def test_trained_BN_on_stats(bn, t_name):
    queries = {
        "posts": "SELECT COUNT(*) FROM posts as p WHERE posts.CommentCount<=18 AND posts.CreationDate>='2010-07-23 07:27:31'::timestamp AND posts.CreationDate<='2014-09-09 01:43:00'::timestamp",
        "comments": "SELECT COUNT(*) FROM comments as c WHERE comments.CreationDate>='2010-08-05 00:36:02'::timestamp AND comments.CreationDate<='2014-09-08 16:50:49'::timestamp",
        "postHistory": "SELECT COUNT(*) FROM postHistory as ph WHERE postHistory.PostHistoryTypeId=1 AND postHistory.CreationDate>='2010-09-14 11:59:07'::timestamp",
        "votes": "SELECT COUNT(*) FROM votes as v WHERE votes.VoteTypeId=2 AND votes.CreationDate<='2014-09-10 00:00:00'::timestamp",
        "postLinks": "SELECT COUNT(*) FROM postLinks as pl WHERE postLinks.LinkTypeId=1 AND postLinks.CreationDate>='2011-09-03 21:00:10'::timestamp AND postLinks.CreationDate<='2014-07-30 21:29:52'::timestamp",
        "users": "SELECT COUNT(*) FROM users as u WHERE users.DownVotes>=0 AND users.DownVotes<=0 AND users.UpVotes>=0 AND users.UpVotes<=31 AND users.CreationDate<='2014-08-06 20:38:52'::timestamp",
        "badges": "SELECT COUNT(*) FROM badges as b WHERE badges.Date>='2010-09-26 12:17:14'::timestamp",
        "tags": "SELECT COUNT(*) FROM tags"
    }

    true_cards = {
        "posts": 90764,
        "comments": 172156,
        "postHistory": 42308,
        "votes": 261476,
        "postLinks": 8776,
        "users": 37062,
        "badges": 77704,
        "tags": 1032
    }

    bn.init_inference_method()
    bn.infer_algo = "exact-jit"
    query = parse_query_single_table(queries[t_name], bn)
    pred = bn.query(query)
    assert min(pred, true_cards[t_name]) / max(pred, true_cards[t_name]) <= 1.5, f"Qerror too large, we have predition" \
                                                                        f"{pred} for true card {true_cards[t_name]}"

    query = parse_query_single_table(queries[t_name], bn)
    _, id_probs = bn.query_id_prob(query, bn.id_attributes)
    if t_name not in ['votes', 'tags']:
        assert min(pred, np.sum(id_probs)) / max(pred, np.sum(id_probs)) <= 1.5, "query_id_prob is incorrect"


data_path = "~/End-to-End-CardEst-Benchmark/datasets/stats_simplified/{}.csv"
model_folder = "/home/ubuntu/data_CE/saved_models"
n_bins = 200
bucket_method = "greedy"
save_bucket_bins = False

data, null_values, key_attrs, table_buckets, equivalent_keys, schema, bin_size = process_stats_data(data_path,
                                                            model_folder, n_bins, bucket_method, save_bucket_bins)
all_bns = dict()
for table in schema.tables:
    t_name = table.table_name
    bn = Bayescard_BN(t_name, key_attrs[t_name], bin_size[t_name], null_values=null_values[t_name])
    bn.build_from_data(data[t_name])
    test_trained_BN_on_stats(bn, t_name)
    all_bns[t_name] = bn
    #model_path = model_folder + f"{t_name}.pkl"
    #pickle.dump(bn, open(model_path, 'wb'), pickle.HIGHEST_PROTOCOL)

be = Bound_ensemble(all_bns, table_buckets, schema)

In [ ]:
model_folder = "/home/ubuntu/data_CE/CE_scheme_models/"
model_path = model_folder + f"model_stats_greedy_{n_bins}.pkl"
pickle.dump(be, open(model_path, 'wb'), pickle.HIGHEST_PROTOCOL)

In [ ]:
model_path = "/home/ubuntu/data_CE/CE_scheme_models/model_stats_greedy_200.pkl"
with open(model_path, "rb") as f:
    bound_ensemble = pickle.load(f)

In [ ]:
from Join_scheme.join_graph import process_condition, get_join_hyper_graph
from BayesCard.Evaluation.cardinality_estimation import timestamp_transorform


OPS = {
    '>': np.greater,
    '<': np.less,
    '>=': np.greater_equal,
    '<=': np.less_equal,
    '=': np.equal,
    '==': np.equal
}


def construct_table_query(BN, table_query, attr, ops, val, epsilon=1e-6):
    if BN is None or attr not in BN.attr_type:
        return None
    if BN.attr_type[attr] == 'continuous':
        if ops == ">=":
            query_domain = (val, np.infty)
        elif ops == ">":
            query_domain = (val + epsilon, np.infty)
        elif ops == "<=":
            query_domain = (-np.infty, val)
        elif ops == "<":
            query_domain = (-np.infty, val - epsilon)
        elif ops == "=" or ops == "==":
            query_domain = val
        else:
            assert False, f"operation {ops} is invalid for continous domain"

        if attr not in table_query:
            table_query[attr] = query_domain
        else:
            prev_l = table_query[attr][0]
            prev_r = table_query[attr][1]
            query_domain = (max(prev_l, query_domain[0]), min(prev_r, query_domain[1]))
            table_query[attr] = query_domain

    else:
        attr_domain = BN.domain[attr]
        if type(attr_domain[0]) != str:
            attr_domain = np.asarray(attr_domain)
        if ops == "in":
            assert type(val) == list, "use list for in query"
            query_domain = val
        elif ops == "=" or ops == "==":
            if type(val) == list:
                query_domain = val
            else:
                query_domain = [val]
        else:
            if type(val) == list:
                assert len(val) == 1
                val = val[0]
                assert (type(val) == int or type(val) == float)
            operater = OPS[ops]
            query_domain = list(attr_domain[operater(attr_domain, val)])

        if attr not in table_query:
            table_query[attr] = query_domain
        else:
            query_domain = [i for i in query_domain if i in table_query[attr]]
            table_query[attr] = query_domain

    return table_query


def parse_query_simple(bns, query):
    """
    If your selection query contains no aggregation and nested sub-queries, you can use this function to parse a
    join query. Otherwise, use parse_query function.
    """
    query = query.replace(" where ", " WHERE ")
    query = query.replace(" from ", " FROM ")
    query = query.replace(" and ", " AND ")
    query = query.split(";")[0]
    query = query.strip()
    tables_all = {}
    join_cond = []
    table_query = {}
    join_keys = {}
    tables_str = query.split(" WHERE ")[0].split(" FROM ")[-1]
    for table_str in tables_str.split(","):
        table_str = table_str.strip()
        if " as " in table_str:
            tables_all[table_str.split(" as ")[-1]] = table_str.split(" as ")[0]
        else:
            tables_all[table_str.split(" ")[-1]] = table_str.split(" ")[0]

    # processing conditions
    conditions = query.split(" WHERE ")[-1].split(" AND ")
    for cond in conditions:
        table, cond, join, join_key = process_condition(cond, tables_all)
        if not join:
            attr = cond[0]
            op = cond[1]
            value = cond[2]
            if "Date" in attr:
                assert "::timestamp" in value
                value = timestamp_transorform(value.strip().split("::timestamp")[0])
            if table not in table_query:
                table_query[table] = dict()
            construct_table_query(bns[table], table_query[table], attr, op, value)
        else:
            join_cond.append(cond)
            for tab in join_key:
                if tab in join_keys:
                    join_keys[tab].add(join_key[tab])
                else:
                    join_keys[tab] = set([join_key[tab]])

    return tables_all, table_query, join_cond, join_keys

In [ ]:
true_card = 17849233970
query57 = "SELECT COUNT(*) FROM posts as p, postLinks as pl, postHistory as ph, votes as v, badges as b, users as u WHERE p.Id = pl.RelatedPostId AND u.Id = p.OwnerUserId AND u.Id = b.UserId AND u.Id = ph.UserId AND u.Id = v.UserId AND p.CommentCount>=0 AND p.CommentCount<=13 AND ph.PostHistoryTypeId=5 AND ph.CreationDate<='2014-08-13 09:20:10'::timestamp AND v.CreationDate>='2010-07-19 00:00:00'::timestamp AND b.Date<='2014-09-09 10:24:35'::timestamp AND u.Views>=0 AND u.DownVotes>=0 AND u.CreationDate>='2010-08-04 16:59:53'::timestamp AND u.CreationDate<='2014-07-22 15:15:22'::timestamp;"
t = time.time()
tables_all, table_cond, join_cond, join_keys = parse_query_simple(bound_ensemble.bns, query57)

print(time.time()-t)

In [ ]:
join_keys["posts"]

In [ ]:
a, b = bound_ensemble.bns['posts'].query_id_prob(table_cond['posts'], list(join_keys["posts"]))

In [ ]:
query1 = "SELECT COUNT(*) FROM posts as p WHERE posts.CommentCount<=18 AND posts.CreationDate>='2010-07-23 07:27:31'::timestamp AND posts.CreationDate<='2014-09-09 01:43:00'::timestamp"
print("true1: 90764")
query2 = "SELECT COUNT(*) FROM comments as c WHERE comments.CreationDate>='2010-08-05 00:36:02'::timestamp AND comments.CreationDate<='2014-09-08 16:50:49'::timestamp"
print("true2: 172156")
query3 = "SELECT COUNT(*) FROM postHistory as ph WHERE postHistory.PostHistoryTypeId=1 AND postHistory.CreationDate>='2010-09-14 11:59:07'::timestamp"
print("true3: 42308")
query4 = "SELECT COUNT(*) FROM votes as v WHERE votes.VoteTypeId=2 AND votes.CreationDate<='2014-09-10 00:00:00'::timestamp"
print("true4: 261476")
query5 = "SELECT COUNT(*) FROM postLinks as pl WHERE postLinks.LinkTypeId=1 AND postLinks.CreationDate>='2011-09-03 21:00:10'::timestamp AND postLinks.CreationDate<='2014-07-30 21:29:52'::timestamp"
print("true5: 8776")
query6 = "SELECT COUNT(*) FROM users as u WHERE users.DownVotes>=0 AND users.DownVotes<=0 AND users.UpVotes>=0 AND users.UpVotes<=31 AND users.CreationDate<='2014-08-06 20:38:52'::timestamp"
print("true6: 37062")
query7 = "SELECT COUNT(*) FROM badges as b WHERE badges.Date>='2010-09-26 12:17:14'::timestamp"
print("true7: 77704")
query8 = "SELECT COUNT(*) FROM tags"
print("true8: 1032")



In [ ]:
bn.init_inference_method()
bn.infer_algo = "exact-jit"
query = parse_query_single_table(query2, bn)
id_attrs = bn.id_attributes
t = time.time()
res = bn.query_id_prob(query, id_attrs)
print(np.sum(res[-1]))
print(time.time()-t)
query = parse_query_single_table(query2, bn)
t = time.time()
print(bn.query(query))
print(time.time() - t)

In [ ]:
bn.id_attributes

In [ ]:
bn.encoding['posts.OwnerUserId'].astype(int)

In [ ]:
from BayesCard.Evaluation.cardinality_estimation import parse_query_single_table
def debug_init_inference_method(bn):
    cpds, topological_order, topological_order_node = bn.align_cpds_in_topological()
    bn.cpds = cpds
    bn.topological_order = topological_order
    bn.topological_order_node = topological_order_node
    bn.infer_machine = VariableEliminationJIT(bn.model, cpds, topological_order, topological_order_node,
                                            fanouts=bn.id_attributes)

debug_init_inference_method(bn)
query, n_distinct = bn.query_decoding(parse_query_single_table(query3, bn))
id_attrs = ['postHistory.PostId', 'postHistory.UserId']
t = time.time()
res = bn.infer_machine.temp(query, id_attrs, n_distinct, bn)
print(time.time()-t)

In [ ]:
query, n_distinct = bn.query_decoding(parse_query_single_table(query3, bn))
bn.infer_machine.query(query, n_distinct) * bn.nrows

In [ ]:
np.sum(res[-1])

In [ ]:
res

In [ ]:
res[-1].shape

In [ ]:
bn.bin_size

In [ ]:
bn.id_exist_null

In [ ]:
query = "SELECT COUNT(*) FROM comments as c WHERE comments.CreationDate>='2010-08-05 00:36:02'::timestamp AND comments.CreationDate<='2014-09-08 16:50:49'::timestamp"
query, n_distinct = bn.query_decoding(parse_query_single_table(query, bn))
t = time.time()
print(bn.infer_machine.query(query) * bn.nrows)
print(time.time() - t)

In [ ]:
a = np.ones((3,4))
a.shape

In [ ]:
a[[1,2]][:, [2,3]] = 0

In [ ]:
a[np.ix_([1,2], [2,3])] = 0

In [ ]:
a

In [ ]:
b = np.ones(3)
(a.T * b).T.shape